In [1]:
import epimodel
import numpy as np
import pandas as pd
import yaml

## New gleam regions, add countries to continents, gleam IDs to countries

In [ ]:
d=pd.read_csv('../../data/sources/md_cities.tsv', delimiter='\t')

In [ ]:
rds=epimodel.RegionDataset.load('../../data/regions.csv') 
nd=[] 
codes=[]
for _, a in d.iterrows(): 
    if a['Country name'] in {'Kosovo', 'Cyprus, Northern'}: 
        continue 
    rc = rds.find_one_by_name(a['Country name'], levels='country') 
    rcnt = rds.find_one_by_name(a['Continent name'], levels='continent') 
    nd.append([a['City Name'], '', '', 'gleam_basin', '', rcnt.Code, '', rc.Code, rc.CountryCodeISOa3, '', np.nan, np.nan, np.nan, a['City ID']]) 
    codes.append(f"G-{a['Airport code']}")
    rds.data.loc[rc.Code, 'GleamID'] = a['Country ID'] 
    rds.data.loc[rcnt.Code, 'GleamID'] = a['Continent ID'] 
    rds.data.loc[rc.Code, 'ContinentCode'] = rcnt.Code 

In [ ]:
sorted(codes)

In [ ]:
rds.write_csv('../../data/regions-new.csv')

In [ ]:
grds=epimodel.RegionDataset()
gd = pd.DataFrame(nd, columns=list(rds.COLUMN_TYPES.keys()), index=pd.Index(codes, name="Code"))
grds.data = grds.data.append(gd)
grds.data
grds._rebuild_index()
grds.write_csv('../../data/regions-gleam.csv')

## Update populations in gleam regs.

In [2]:
grds = epimodel.RegionDataset.load('../../data/regions-gleam.csv')

In [9]:
with open("../../data/sources/regions.yaml") as f: y = yaml.safe_load(f)

In [10]:
def rec(yr):
    if yr['kind'] == 'city':
        name = yr['names'][0]
        if name.endswith('city'):
            name = name[:-5]
        gr = grds.find_all_by_name(name)
        if len(gr)!=1:
            print('EE', yr['names'], gr)
        else:
            gr = gr[0]
            if 'population' in yr:
                grds.data.loc[gr.Code, 'Population'] = yr['population']
            if 'lat' in yr:
                grds.data.loc[gr.Code, 'Lat'] = yr['lat']
                grds.data.loc[gr.Code, 'Lon'] = yr['lon']
    for i in yr.get('subregions', []):
        rec(i)
rec(y)

EE ['Ercan'] ()
EE ['Pristina'] ()


In [11]:
grds.write_csv('../../data/regions-gleam.csv')